In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import math
from tqdm import tqdm 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/Shareddrives/CSCI\ 724
%ls

/content/drive/Shareddrives/CSCI 724
Code.ipynb      serviceTpAvg.p       userServiceDictTp.p
dataset2/       userRtAvg.p          user_similarity_matrix_Response_Time.p
serviceRtAvg.p  userServiceDictRt.p  userTpAvg.p


In [3]:
rtdata2 = pd.read_csv('dataset2/rtdata.txt',' ',names=['User ID','Service ID','Time Slice ID','Response Time (sec)'])
tpdata2 = pd.read_csv('dataset2/tpdata.txt',' ',names=['User ID','Service ID','Time Slice ID','Throughput (kbps)'])

In [4]:
rtdata2.head()

,User ID,Service ID,Time Slice ID,Response Time (sec)
0,0,0,0,4.180
1,0,1,0,0.416
2,0,2,0,0.441
3,0,3,0,2.764
4,0,4,0,0.465


In [5]:
tpdata2.head()

,User ID,Service ID,Time Slice ID,Throughput (kbps)
0,0,0,0,0.086842
1,0,1,0,1.637019
2,0,2,0,1.902494
3,0,3,0,0.131331
4,0,4,0,0.780645


In [6]:
userct = rtdata2['User ID'].nunique()
servicect = rtdata2['Service ID'].nunique()
tb_current = max(rtdata2['Time Slice ID'].max(),tpdata2['Time Slice ID'].max())
print(userct,servicect,tb_current)

142 4500 63


What I want to do here is for each user, for each service get the array of  tuples of time slice ID and response Time & 

for each service, for each user get the array of  tuples of time slice ID and response Time

In [7]:
userServiceDictRt = pickle.load( open( "userServiceDictRt.p", "rb" ) )

In [ ]:
userServiceDictRt = rtdata2.groupby(['User ID','Service ID']).apply(func=lambda x: x[['Time Slice ID','Response Time (sec)']].to_numpy()).to_dict()
userServiceDictRt[(0,0)]

In [ ]:
pickle.dump( userServiceDictRt, open( "userServiceDictRt.p", "wb" ))

What I want to do here is for each user, for each service get the array of  tuples of time slice ID and Throughput & 

for each service, for each user get the array of  tuples of time slice ID and Throughput

In [8]:
userServiceDictTp = pickle.load( open( "userServiceDictTp.p", "rb" ) )

In [ ]:
userServiceDictTp = tpdata2.groupby(['User ID','Service ID']).apply(func=lambda x: x[['Time Slice ID','Throughput (kbps)']].to_numpy()).to_dict()
userServiceDictTp[(0,0)]

In [ ]:
userServiceDictTp[(0,0)]

In [ ]:
pickle.dump( userServiceDictTp, open( "userServiceDictTp.p", "wb" ))

Get Average Response Time Value for users and services

In [9]:
userRtAvg = pickle.load(open( "userRtAvg.p", "rb" ))
serviceRtAvg = pickle.load(open( "serviceRtAvg.p", "rb" ))

In [ ]:
userRtAvg = rtdata2[['User ID','Response Time (sec)']].groupby(['User ID']).agg('mean').to_numpy()[:,0] #apply(func=lambda x: x[['Time Slice ID','Throughput (kbps)']].to_numpy()).to_dict()
serviceRtAvg = rtdata2[['Service ID','Response Time (sec)']].groupby(['Service ID']).agg('mean').to_numpy()[:,0]
userRtAvg,serviceRtAvg

In [ ]:
pickle.dump(userRtAvg, open("userRtAvg.p", "wb"))
pickle.dump(serviceRtAvg, open("serviceRtAvg.p", "wb"))

Get Average Throughput Value for users and services

In [10]:
userTpAvg = pickle.load(open( "userTpAvg.p", "rb" ))
serviceTpAvg = pickle.load(open( "serviceTpAvg.p", "rb" ))

In [ ]:
userTpAvg = tpdata2[['User ID','Throughput (kbps)']].groupby(['User ID']).agg('mean').to_numpy()[:,0] #apply(func=lambda x: x[['Time Slice ID','Throughput (kbps)']].to_numpy()).to_dict()
serviceTpAvg = tpdata2[['Service ID','Throughput (kbps)']].groupby(['Service ID']).agg('mean').to_numpy()[:,0]
userTpAvg,serviceTpAvg

In [ ]:
pickle.dump(userTpAvg, open("userTpAvg.p", "wb"))
pickle.dump(serviceTpAvg, open("serviceTpAvg.p", "wb"))

Compute Similarity

In [11]:
def getCoInvokedServices(dic, uid1,uid2):
  services1 = []
  services2 = []

  services1_unique_ct = 0
  services2_unique_ct = 0
  
  for service_idx in range(servicect):
    if (uid1,service_idx) in dic and (uid2,service_idx) in dic:
      services1.append((uid1,service_idx))
      services2.append((uid2,service_idx))

    elif (uid1,service_idx) in dic:
      services1_unique_ct+=1
    elif (uid2,service_idx) in dic:
      services2_unique_ct+=1
  
  return services1,services2,services1_unique_ct,services2_unique_ct

In [12]:
def getCommonUsers(dic, serv1,serv2):
  users1 = []
  users2 = []

  users1_unique_ct = 0
  users2_unique_ct = 0

  for user_idx in range(userct):
    if (user_idx,serv1) in dic and (user_idx,serv2) in dic:
      users1.append((user_idx,serv1))
      users2.append((user_idx,serv2))

    elif (user_idx,serv1) in dic:
      users1_unique_ct+=1
    elif (user_idx,serv2) in dic:
      users2_unique_ct+=1

  return users1,users2,users1_unique_ct,users2_unique_ct

In [13]:
def getSingleItemCalculations(qos1, qos2, q_avg1, q_avg2, t_current, alpha, beta):
  
  diff1 = qos1[:,1] - q_avg1
  diff2 = qos2[:,1] - q_avg2
  
  time1 = qos1[:,0]
  time2 = qos2[:,0]

  den1 = (diff1*diff1).mean()
  den2 = (diff2*diff2).mean()

  diff1_rep = diff1.repeat(len(diff2)).reshape(len(diff1),len(diff2))
  diff2_rep = diff2.repeat(len(diff1)).reshape(len(diff2),len(diff1))

  diff_prod = diff1_rep.transpose()*diff2_rep

  time1_rep = time1.repeat(len(time2)).reshape(len(time1),len(time2))
  time2_rep = time2.repeat(len(time1)).reshape(len(time2),len(time1))

  decay1 = np.exp(-alpha * np.abs(time1_rep.transpose()-time2_rep))
  decay2 = np.exp(-beta * np.abs(t_current - (time1_rep.transpose()+time2_rep)/2))

  num1 = (diff_prod*decay1*decay2).mean()

  del diff1,diff2,time1,time2,diff1_rep,diff2_rep,diff_prod,time1_rep,time2_rep,decay1,decay2

  return num1, den1, den2

In [14]:
def getSimilarity(dic, user_flag, id1, id2, q_avg1, q_avg2, t_current, alpha, beta):
  if user_flag:
    common_item1, common_item2, unique_ct1, unique_ct2  = getCoInvokedServices(dic, id1, id2)
  else:
    common_item1, common_item2, unique_ct1, unique_ct2 = getCommonUsers(dic, id1, id2)
  
  commonality = len(common_item1)

  if commonality==0:
    return 0

  num1_sum, den1_sum, den2_sum = 0, 0, 0
  
  for idx in range(commonality):
    (num1, den1, den2) = getSingleItemCalculations(dic[common_item1[idx]], dic[common_item2[idx]], q_avg1, q_avg2, t_current, alpha, beta)
    num1_sum+=num1
    den1_sum+=den1
    den2_sum+=den2
  
  w = num1_sum/((den1_sum*den2_sum)**0.5)
  w = 2*(commonality)*w/((commonality+unique_ct1)+(commonality+unique_ct2))

  del common_item1, common_item2, commonality    
  return w

In [15]:
def computeSimilarityMatrix(dic, user_flag, qos_avg, t_current, alpha, beta, QOS_type):
  if user_flag:
    ct = userct
    str_name = 'user'
  else:
    ct = servicect
    str_name = 'service'
  qos_matrix = np.zeros((ct,ct))


  outer = np.array(list(range(ct)))
  inner = np.array(list(range(ct)))
  cartesian = np.transpose([np.tile(outer, len(inner)), np.repeat(inner, len(outer))])
  

  for idx in tqdm(range(len(cartesian)), desc='User Spaces'):
    id1,id2 = cartesian[idx][0],cartesian[idx][1]
    if id1<id2: 
      qos_matrix[id1][id2] = getSimilarity(dic, user_flag, id1, id2, qos_avg[id1], qos_avg[id2], t_current, alpha, beta)
      qos_matrix[id2][id1] = qos_matrix[id1][id2]
  pickle.dump(qos_matrix, open(str_name+"_similarity_matrix_"+QOS_type+".p", "wb"))
  return qos_matrix

In [ ]:
ans = computeSimilarityMatrix(dic = userServiceDictRt, user_flag = True, qos_avg = userRtAvg, t_current = tb_current, alpha=1, beta=1, QOS_type = "Response_Time")

Streaming output truncated to the last 5000 lines.
User Spaces: 100%|██████████| 20164/20164 [1:21:00<00:00,  4.15it/s]


In [ ]:
ans = computeSimilarityMatrix(dic = userServiceDictRt, user_flag = False, qos_avg = serviceRtAvg, t_current = tb_current, alpha=1, beta=1, QOS_type = "Response_Time")

User Spaces:  39%|███▊      | 7809230/20250000 [6:43:08<61:17:39, 56.38it/s]

In [ ]:
def similarityMatrixReconstruction(qos_matrix, d):
  qos_mat = np.maximum(0,qos_matrix)
  pre_computed = (1-d)*np.linalg.inv(1 - d*qos_mat)
  